# My try for Part 2A in Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [9]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))
print(G.nodes(data=True))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458
[(0, {'Department': 1, 'ManagementSalary': 0.0}), (1, {'Department': 1, 'ManagementSalary': nan}), (2, {'Department': 21, 'ManagementSalary': nan}), (3, {'Department': 21, 'ManagementSalary': 1.0}), (4, {'Department': 21, 'ManagementSalary': 1.0}), (5, {'Department': 25, 'ManagementSalary': nan}), (6, {'Department': 25, 'ManagementSalary': 1.0}), (7, {'Department': 14, 'ManagementSalary': 0.0}), (8, {'Department': 14, 'ManagementSalary': nan}), (9, {'Department': 14, 'ManagementSalary': 0.0}), (10, {'Department': 9, 'ManagementSalary': 0.0}), (11, {'Department': 14, 'ManagementSalary': 0.0}), (12, {'Department': 14, 'ManagementSalary': 1.0}), (13, {'Department': 26, 'ManagementSalary': 1.0}), (14, {'Department': 4, 'ManagementSalary': nan}), (15, {'Department': 17, 'ManagementSalary': 0.0}), (16, {'Department': 34, 'ManagementSalary': 0.0}), (17, {'Department': 1, 'ManagementSalary': 0.0}), (18, {'

## Convert the graph information into dataframe

In [13]:
df = pd.DataFrame(index=G.nodes())

df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))

In [45]:
# Assign different measurements into dataframe
df['degree'] = pd.Series(nx.degree_centrality(G))
df['clustering'] = pd.Series(nx.clustering(G))
df['closeness'] = pd.Series(nx.closeness_centrality(G))
df['betweenness'] = pd.Series(nx.betweenness_centrality(G))

df.head()

,ManagementSalary,Department,degree,closeness,betweenness,clustering
0,0.0,1,0.043825,0.421991,0.001124,0.276423
1,NaN,1,0.051793,0.422360,0.001195,0.265306
2,NaN,21,0.094622,0.461490,0.006570,0.297803
3,1.0,21,0.070717,0.441663,0.001654,0.384910
4,1.0,21,0.095618,0.462152,0.005547,0.318691


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1005 entries, 0 to 1004
Data columns (total 6 columns):
ManagementSalary    753 non-null float64
Department          1005 non-null int64
degree              1005 non-null float64
closeness           1005 non-null float64
betweenness         1005 non-null float64
clustering          1005 non-null float64
dtypes: float64(5), int64(1)
memory usage: 95.0 KB


In [42]:
# check the count of NaN in ManagementSalary
df.isnull().sum()

ManagementSalary    252
Department            0
degree                0
closeness             0
betweenness           0
clustering            0
dtype: int64

In [55]:
# Split the dataframe into two sub-dataframes: df_train & df_pred
df_train = df.dropna()

df_train.head()

,ManagementSalary,Department,degree,closeness,betweenness,clustering
0,0.0,1,0.043825,0.421991,0.001124,0.276423
3,1.0,21,0.070717,0.441663,0.001654,0.384910
4,1.0,21,0.095618,0.462152,0.005547,0.318691
6,1.0,25,0.114542,0.475805,0.012387,0.155183
7,0.0,14,0.071713,0.420156,0.002818,0.287785


In [56]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 0 to 1004
Data columns (total 6 columns):
ManagementSalary    753 non-null float64
Department          753 non-null int64
degree              753 non-null float64
closeness           753 non-null float64
betweenness         753 non-null float64
clustering          753 non-null float64
dtypes: float64(5), int64(1)
memory usage: 41.2 KB


In [81]:
df_pred = df[np.isnan(df['ManagementSalary'])]

df_pred.head()

,ManagementSalary,Department,degree,closeness,betweenness,clustering
1,NaN,1,0.051793,0.422360,0.001195,0.265306
2,NaN,21,0.094622,0.461490,0.006570,0.297803
5,NaN,25,0.170319,0.501484,0.030995,0.107002
8,NaN,14,0.036853,0.413151,0.000557,0.447059
14,NaN,4,0.079681,0.442068,0.003726,0.215784


In [82]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 1 to 1001
Data columns (total 6 columns):
ManagementSalary    0 non-null float64
Department          252 non-null int64
degree              252 non-null float64
closeness           252 non-null float64
betweenness         252 non-null float64
clustering          252 non-null float64
dtypes: float64(5), int64(1)
memory usage: 13.8 KB


## Prepare training dataseet

In [130]:
# selected_attrs = ['Department', 'degree', 'closeness', 'betweenness', 'clustering']
selected_attrs = ['degree', 'closeness', 'betweenness', 'clustering']
X = df_train[selected_attrs]

X.head()

,degree,closeness,betweenness,clustering
0,0.043825,0.421991,0.001124,0.276423
3,0.070717,0.441663,0.001654,0.384910
4,0.095618,0.462152,0.005547,0.318691
6,0.114542,0.475805,0.012387,0.155183
7,0.071713,0.420156,0.002818,0.287785


In [131]:
y = df_train['ManagementSalary']

# y

In [132]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Scale the X data

In [133]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_pred = scaler.transform(df_pred[selected_attrs])

## Training with Logistic Regression

In [134]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [135]:
# model score
# clf.score(X_train, y_train)

In [136]:
# predict the X_test
pred1 = clf.predict(X_test)
pred2 = clf.predict_proba(X_test)

# pred2

## Test Training dataset

In [137]:
from sklearn import metrics

# Accuracy
metrics.accuracy_score(y_test, pred1)

0.93377483443708609

In [138]:
# AUC
metrics.roc_auc_score(y_test, pred2[:, 1])

0.93516483516483517

In [139]:
# Confusion
metrics.confusion_matrix(y_test, pred1)

array([[130,   0],
       [ 10,  11]], dtype=int64)

In [140]:
print(metrics.classification_report(y_test, pred1))

             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       130
        1.0       1.00      0.52      0.69        21

avg / total       0.94      0.93      0.92       151



## Generate result for Prediction dataset

In [141]:
pred = clf.predict_proba(df_pred[selected_attrs])

pred[:, 1]

array([ 0.18185786,  0.20806357,  0.1825705 ,  0.22533044,  0.17831979,
        0.20006833,  0.21210888,  0.22735825,  0.21760724,  0.26602867,
        0.2089846 ,  0.20229273,  0.20778333,  0.25066163,  0.25966727,
        0.20850796,  0.19935774,  0.17830973,  0.21616723,  0.20141974,
        0.2418947 ,  0.18327289,  0.2220506 ,  0.19565825,  0.20602426,
        0.17786934,  0.19164654,  0.20768742,  0.24406615,  0.23353264,
        0.23314831,  0.25563047,  0.21280899,  0.22569054,  0.20102434,
        0.30600803,  0.25273648,  0.21959529,  0.19796067,  0.23181445,
        0.20481375,  0.19087077,  0.26650509,  0.16795515,  0.23917934,
        0.21046328,  0.2532424 ,  0.2376626 ,  0.21817364,  0.25347495,
        0.2065373 ,  0.21279212,  0.22948657,  0.23821997,  0.18918399,
        0.2288499 ,  0.19990487,  0.21767575,  0.25839208,  0.20457914,
        0.17969555,  0.2134119 ,  0.25579988,  0.21460848,  0.21441727,
        0.18057217,  0.18455341,  0.17667831,  0.20313855,  0.21

In [ ]:
# resukt with `Department` column
array([ 0.18871593,  0.40575873,  0.41107841,  0.3543703 ,  0.206628  ,
        0.2072682 ,  0.30722262,  0.32628061,  0.31411355,  0.37307983,
        0.30323337,  0.29474868,  0.36225472,  0.47574454,  0.48748899,
        0.26560358,  0.56190237,  0.20666887,  0.23979476,  0.22407591,
        0.39715916,  0.31742552,  0.2814145 ,  0.25919868,  0.2900689 ,
        0.17785943,  0.30956201,  0.57433735,  0.66550078,  0.59940935,
        0.2942118 ,  0.62747889,  0.39066712,  0.55525753,  0.2568516 ,
        0.53262421,  0.29719023,  0.25192967,  0.2282176 ,  0.27417205,
        0.20456464,  0.19792104,  0.50763223,  0.16807832,  0.42675665,
        0.38731117,  0.44516428,  0.37011757,  0.34506434,  0.37934631,
        0.39259962,  0.52551576,  0.28073843,  0.32950953,  0.18909467,
        0.35892427,  0.32071727,  0.34428339,  0.29384917,  0.47907908,
        0.32217086,  0.2130707 ,  0.49386841,  0.5843741 ,  0.24629381,
        0.23263853,  0.30945092,  0.29827829,  0.27728125,  0.21780365,
        0.22955821,  0.19787807,  0.21541426,  0.18654659,  0.18079151,
        0.21351059,  0.55273965,  0.26840327,  0.53752841,  0.18492121,
        0.20617147,  0.20715916,  0.30702401,  0.34848215,  0.59349507,
        0.56139229,  0.32686728,  0.5905656 ,  0.27227942,  0.43832209,
        0.31671666,  0.40050513,  0.31585588,  0.27173832,  0.29429539,
        0.2610346 ,  0.17752409,  0.36244926,  0.41972544,  0.38619797,
        0.40243265,  0.43260934,  0.55237561,  0.19354556,  0.29666111,
        0.37020308,  0.32279044,  0.6033507 ,  0.23608486,  0.25203364,
        0.34051635,  0.31361778,  0.5015679 ,  0.39713621,  0.18181351,
        0.20595921,  0.25442484,  0.30537252,  0.31737084,  0.64505104,
        0.5431292 ,  0.33241359,  0.38538877,  0.40088238,  0.33304462,
        0.52204671,  0.2492306 ,  0.5876637 ,  0.44038213,  0.27399044,
        0.12042879,  0.34975744,  0.25821973,  0.24672074,  0.5305864 ,
        0.27128895,  0.31815108,  0.24333478,  0.47563764,  0.24883155,
        0.36980305,  0.09544034,  0.32860899,  0.44281734,  0.08919036,
        0.56198225,  0.14228007,  0.54091642,  0.59726909,  0.22545598,
        0.37542951,  0.43374424,  0.60741732,  0.26420722,  0.40640992,
        0.21131055,  0.1969698 ,  0.17739178,  0.35899759,  0.3231519 ,
        0.10148088,  0.4787775 ,  0.43967994,  0.28447019,  0.23621865,
        0.60026815,  0.46227956,  0.10148088,  0.42870558,  0.33295391,
        0.52079881,  0.39423769,  0.34645167,  0.42043487,  0.29981844,
        0.58345421,  0.53293464,  0.39415114,  0.2500075 ,  0.33495896,
        0.43166679,  0.51869278,  0.35218377,  0.13330062,  0.37030258,
        0.37340498,  0.47995474,  0.3841464 ,  0.27616663,  0.38571183,
        0.35182244,  0.34334143,  0.22449574,  0.19542668,  0.04343383,
        0.26076296,  0.22697538,  0.09740123,  0.22644609,  0.31798342,
        0.38178966,  0.24288446,  0.21403659,  0.37740601,  0.51892893,
        0.22927099,  0.13245599,  0.13663501,  0.21104208,  0.27808993,
        0.5854471 ,  0.47844719,  0.41545535,  0.09162322,  0.37028972,
        0.19416579,  0.55705007,  0.15012855,  0.60094207,  0.21836035,
        0.52628337,  0.31615335,  0.31020567,  0.35547449,  0.14110859,
        0.31663251,  0.31455386,  0.46901395,  0.40292265,  0.49119754,
        0.19703841,  0.22605547,  0.32465772,  0.43880795,  0.51845155,
        0.35002412,  0.25512827,  0.11467283,  0.13286281,  0.39211235,
        0.51318614,  0.51463312,  0.35463255,  0.48739185,  0.54313096,
        0.43582708,  0.70012253,  0.11969521,  0.22160664,  0.16345247,
        0.27484088,  0.53690241])

## Final Answer

In [ ]:
def salary_predictions(debug=False):
    
    # Your Code Here
    from sklearn import preprocessing
    from sklearn import metrics
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    
    # generate dataset
    df = pd.DataFrame(index=G.nodes())

    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    
    # Assign different measurements into dataframe
    df['degree'] = pd.Series(nx.degree_centrality(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G))
    
    # Split the dataframe into two sub-dataframes: df_train & df_pred
    df_train = df.dropna()
    df_pred = df[np.isnan(df['ManagementSalary'])]
    
#     selected_attrs = ['Department', 'degree', 'closeness', 'betweenness', 'clustering']
    selected_attrs = ['degree', 'closeness', 'betweenness', 'clustering']
    
    X = df_train[selected_attrs]
    y = df_train['ManagementSalary']
    
    # split into training and test data w/ given training dataframe
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
   
    # Scale transformation
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_pred = scaler.transform(df_pred[selected_attrs])
    
    # Traing data with Logistic Regression Model
    clf = LogisticRegression()

    clf.fit(X_train, y_train)
    
    # check with training dataset with various metrics
    if debug:
        # predict the X_test
        pred1 = clf.predict(X_test)
        pred2 = clf.predict_proba(X_test)
        
        print("\nAccuracy: {}".format(metrics.accuracy_score(y_test, pred1)))
        print("\nROC_AUC: {}".format(metrics.roc_auc_score(y_test, pred2[:, 1])))
        print("\nConfusiion matrix: \n{}".format(metrics.confusion_matrix(y_test, pred1)))
        print("\nCalssification report: \n{}".format(metrics.classification_report(y_test, pred1)))
        
    # Generte result for autograder
    pred = clf.predict_proba(X_pred)
    
    rlt = pd.Series(data=pred[:, 1], index=df_pred.index)
    
    return rlt # Your Answer Here

# salary_predictions(debug=True)